In [1]:
from qtpg.team import Team
from qtpg.learner import Learner
from qtpg.program import Program
from qtpg.rule import Rule
from qtpg.figure13 import Figure13
from qtpg.search_manager import SearchManager
import numpy as np
import uuid
import copy

In [2]:
# env params
memorySize = 20
legalMove = 0.1
illegalMove = -0.01
outOfBounds = -0.01
memoryRepeat = 0 #-0.01
goalReached = 1000

env = Figure13(5, 5, (2, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)

In [6]:
adam_and_eve = []

init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], [2, 3], 0)
init_learner = Learner(uuid.uuid4(), init_rule)
(init_team.learners).append(init_learner)
init_team.mostRecent = init_learner
adam_and_eve.append(init_team)

for adam in adam_and_eve:
    print(adam.learners)
    print('------')
    win = adam.search(env)
    print(adam.learners)
    for learner in adam.learners:
        print(learner.program.rule.action_set)
        print(learner.program.rule.region)

------
Sample start: [0, 1]
[<qtpg.learner.Learner object at 0x7f9ec3025a90>, <qtpg.learner.Learner object at 0x7f9ec3025d00>]
[2, 3]
[0, 0, 2, 0]
[0, 1]
[1, 1, 1, 1]


In [ ]:
numGens = 50
maxTeamPool = 5
# init search manager and starting teams
searchManager = SearchManager(maxTeamPool)

# init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
# init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
# init_learner = Learner(uuid.uuid4(), init_rule)
# (init_team.learners).append(init_learner)
# init_team.mostRecent = init_learner
# (searchManager.teamPool).append(init_team)

# init teams
adam_and_eve = []

init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], [0, 1], 0)
init_learner = Learner(uuid.uuid4(), init_rule)
(init_team.learners).append(init_learner)
init_team.mostRecent = init_learner
adam_and_eve.append(init_team)

# print('-----init pop start-----')
# for team in searchManager.teamPool:
#     print(f'Team id: {team.id}')
#     print('Learners:')
#     for learner in team.learners:
#         print(learner.id)
# print('-----init pop end-----')

for gen in range(numGens):
    toEvaluateAfterGen = []
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} start!>>>>>>>>>>>>>>>>>>>>>>>')
    # get it started
    if gen == 0:
        for adam in adam_and_eve:
            win = adam.search(env)
            data = [adam, win]
            searchManager.evaluate_team(data)
    else:
        for parent in searchManager.teamPool:
            # only run if the team has not yet won
            if parent not in searchManager.winners:
                print(f'Parent: {parent.id}')
                env.current_state = parent.start_state

                # make offspring from parent, then let that offspring make a new rule
                child = copy.deepcopy(parent)
                child.id = uuid.uuid4()
                win = child.search(env)
                print(f'Child: {child.id}')
                print('Start Child Regions --------')
                for learner in child.learners:
                    print(learner.program.rule.region)
                print('End Child Regions --------')
                print(f'Child Fitness: {child.fitness}') 
                toEvaluateAfterGen.append([child, win])
            else:
                print(f'Skipping Team: {parent.id}')
    for data in toEvaluateAfterGen:
        searchManager.evaluate_team(data)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print(f'End of Gen {gen+1} Regions')
    for i in range(len(searchManager.teamPool)):
        print(f'Team: {i+1} --> {searchManager.teamPool[i].fitness}')
        for learner in searchManager.teamPool[i].learners:
            print(learner.program.rule.region)
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} end!>>>>>>>>>>>>>>>>>>>>>>>')
for team in searchManager.winners:
    print('$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$')
    print(team.fitness)
    print('Learners:')
    for learner in team.learners:
        print(f'Region: {learner.program.rule.region} --> Action: {learner.program.rule.action_set}')
    print('$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$')

In [ ]:
for team in searchManager.winners:
    print('$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$')
    print(team.fitness)
    print('Learners:')
    for learner in team.learners:
        print(f'Region: {learner.program.rule.region} --> Action: {learner.program.rule.action_set}')
    print('$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$')